# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [148]:
# import libraries
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import asc, col, datediff, dayofmonth, desc, from_unixtime
from pyspark.sql.functions import max as Fmax
from pyspark.sql.functions import min as Fmin
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType, IntegerType, StringType

%matplotlib inline

In [2]:
# create a Spark session
spark = SparkSession.builder.appName("Sparkify").getOrCreate()

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1591409298092'),
 ('spark.app.name', 'Sparkify'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.port', '42021'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'mint.lan'),
 ('spark.ui.showConsoleProgress', 'true')]

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

## Load Data

In [4]:
file_path = "./mini_sparkify_event_data.json"
df = spark.read.json(file_path)

## Clean Data

### Explore Data

In [5]:
# display columns names
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
# size of the data
df.count(), len(df.columns)

(286500, 18)

In [7]:
# display first 5 record
df.head(5)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

### Clean Data

In [8]:
# make sure we dont have missing values 
df.filter("sessionId is null or userID is null").show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+



In [9]:
# remove data where userID is empty string
df = df.filter("userId != ''")

In [10]:
# shape of cleaned data
df.count(), len(df.columns)

(278154, 18)

# Exploratory Data Analysis



##  Define Churn

In [11]:
# list of events in page column
df.select("page").distinct().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|              Logout|
|            NextSong|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+



We will use the `Cancellation Confirmation` event in the `page` column to define churn, which happen for both paid and free users.

In [170]:
def get_churn(df=df):
    df.createOrReplaceTempView("df")
    # query a table with churned userId and value 1 indicating they are churned user
    churned_userId = spark.sql(
        'SELECT DISTINCT userId, 1 as churn FROM df \
    WHERE page = "Cancellation Confirmation"'
    )
    # query a table with churned userId and value 0 indicating they are churned user
    unchurned_userId = spark.sql(
        'SELECT DISTINCT userId, 0 as churn FROM df \
    WHERE userId NOT IN (SELECT DISTINCT userId from df WHERE Page = "Cancellation Confirmation")'
    )
    # merge two table together
    return churned_userId.union(unchurned_userId)

assert get_churn(df).count() == df.select("userId").distinct().count()

## User Life Time 

In [173]:
def get_lifetime(df=df):
    """
    Returns the lifetime in days since a user has become a registered user
    """
    
    min_ts = df.groupby("userId").agg(Fmin("registration"))
    max_ts = df.groupby("userId").agg(Fmax("ts"))
    ts_df = min_ts.join(max_ts, ["userId"])
    time_df = ts_df.select(
        col("userId"),
        from_unixtime(col("min(registration)") / 1000.0).alias("registration_time"),
        from_unixtime(col("max(ts)") / 1000.0).alias("last_active_time"),
    )
    return time_df.withColumn(
        "lifetime", datediff(col("last_active_time"), col("registration_time"))
    ).select("userId", "lifetime")
assert df.select("userId").distinct().count() == get_lifetime(df).count()

In [174]:
get_lifetime(df).show()

+------+--------+
|userId|lifetime|
+------+--------+
|100010|      56|
|200002|      70|
|   125|      71|
|   124|     132|
|    51|      20|
|     7|      72|
|    15|      56|
|    54|     111|
|   155|      24|
|100014|      85|
|   132|      67|
|   154|      24|
|   101|      54|
|    11|     125|
|   138|      66|
|300017|      74|
|100021|      65|
|    29|      60|
|    69|      71|
|   112|      88|
+------+--------+
only showing top 20 rows



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.